In [ ]:
%matplotlib inline

# Aspectos destacados de la versión para 
# scikit-learn 1.2

Traducido al ESP por: [Luis Pérez](http://OVAT5UG.github.io/)| Abril 10, 2023

.. módulo actual:: sklearn

¡Nos complace anunciar el lanzamiento de scikit-learn 1.2! Muchas correcciones de errores
y se agregaron mejoras, así como algunas características clave nuevas. detallamos
a continuación algunas de las características principales de esta versión. **Para obtener una lista exhaustiva de
todos los cambios**, consulte las `notas de la versión <cambios_1_2>`.

Para instalar la última versión (with pip)::

    pip install --upgrade scikit-learn

or with conda::

    conda install -c conda-forge scikit-learn


## Salida de Pandas con la API `set_output`
Los transformadores de scikit-learn ahora admiten la salida de pandas con la API `set_output`.
Para obtener más información sobre la API `set_output`, consulte el ejemplo:
`sphx_glr_auto_examples_miscellaneous_plot_set_output.py` y
# este [video, salida de pandas DataFrame para transformadores scikit-learn
(algunos ejemplos)](https://youtu.be/5bCg8VfX2x8)_.


In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.compose import ColumnTransformer

X, y = load_iris(as_frame=True, return_X_y=True)
sepal_cols = ["sepal length (cm)", "sepal width (cm)"]
petal_cols = ["petal length (cm)", "petal width (cm)"]

preprocessor = ColumnTransformer(
    [
        ("scaler", StandardScaler(), sepal_cols),
        ("kbin", KBinsDiscretizer(encode="ordinal"), petal_cols),
    ],
    verbose_feature_names_out=False,
).set_output(transform="pandas")

X_out = preprocessor.fit_transform(X)
X_out.sample(n=5, random_state=0)

## Restricciones de interacción en árboles potenciadores de gradiente basados ​​en histogramas
:class:`~ensemble.HistGradientBoostingRegressor` y
:class:`~ensemble.HistGradientBoostingClassifier` ahora admite restricciones de interacción
con el parámetro `interaction_cst`. Para más detalles, consulte el
`Guía del usuario <interaction_cst_hgbt>`. En el siguiente ejemplo, las características no son
permitido interactuar.



In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.ensemble import HistGradientBoostingRegressor

X, y = load_diabetes(return_X_y=True, as_frame=True)

hist_no_interact = HistGradientBoostingRegressor(
    interaction_cst=[[i] for i in range(X.shape[1])], random_state=0
)
hist_no_interact.fit(X, y)

## Pantallas nuevas y mejoradas
:class:`~metrics.PredictionErrorDisplay` proporciona una forma de analizar la regresión
modelos de manera cualitativa.



In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import PredictionErrorDisplay

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
_ = PredictionErrorDisplay.from_estimator(
    hist_no_interact, X, y, kind="actual_vs_predicted", ax=axs[0]
)
_ = PredictionErrorDisplay.from_estimator(
    hist_no_interact, X, y, kind="residual_vs_predicted", ax=axs[1]
)

:class:`~model_selection.LearningCurveDisplay` ahora está disponible para trazar
resultados de :func:`~model_selection.learning_curve`.



In [ ]:
from sklearn.model_selection import LearningCurveDisplay

_ = LearningCurveDisplay.from_estimator(
    hist_no_interact, X, y, cv=5, n_jobs=2, train_sizes=np.linspace(0.1, 1, 5)
)

:class:`~inspection.PartialDependenceDisplay` expone un nuevo parámetro
`categorical_features` para mostrar la dependencia parcial de las características categóricas
utilizando diagramas de barras y mapas de calor.



In [ ]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml(
    "titanic", version=1, as_frame=True, return_X_y=True, parser="pandas"
)
X = X.select_dtypes(["number", "category"]).drop(columns=["body"])

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline

categorical_features = ["pclass", "sex", "embarked"]
model = make_pipeline(
    ColumnTransformer(
        transformers=[("cat", OrdinalEncoder(), categorical_features)],
        remainder="passthrough",
    ),
    HistGradientBoostingRegressor(random_state=0),
).fit(X, y)

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

fig, ax = plt.subplots(figsize=(14, 4), constrained_layout=True)
_ = PartialDependenceDisplay.from_estimator(
    model,
    X,
    features=["age", "sex", ("pclass", "sex")],
    categorical_features=categorical_features,
    ax=ax,
)

## Analizador más rápido en :func:`~datasets.fetch_openml`
:func:`~datasets.fetch_openml` ahora admite un nuevo analizador `"pandas"` que es
más memoria y CPU eficiente. En v1.4, el valor predeterminado cambiará a
`parser="auto"` que usará automáticamente el analizador `"pandas"` para datos densos
data y `"liac-arff"` para datos dispersos.



In [ ]:
X, y = fetch_openml(
    "titanic", version=1, as_frame=True, return_X_y=True, parser="pandas"
)
X.head()

## Compatibilidad con API de matriz experimental en :class:`~discriminant_analysis.LinearDiscriminantAnalysis`
Soporte experimental para [Array API](https://data-apis.org/array-api/latest/)
se agregó la especificación a :class:`~discriminant_analysis.LinearDiscriminantAnalysis`.
El estimador ahora puede ejecutarse en cualquier biblioteca compatible con Array API, como
[CuPy](https://docs.cupy.dev/en/stable/overview.html)_, una matriz acelerada por GPU
biblioteca. Para obtener más información, consulte la `Guía del usuario <array_api>`.



## Eficiencia mejorada de muchos estimadores
En la versión 1.1, la eficiencia de muchos estimadores que se basan en el cálculo de
distancias por pares (esencialmente estimadores relacionados con la agrupación, múltiples
aprendizaje y algoritmos de búsqueda de vecinos) se mejoró mucho para float64
entrada densa. La mejora de la eficiencia fue especialmente una huella de memoria reducida
y una escalabilidad mucho mejor en máquinas multinúcleo.
En la versión 1.2, la eficiencia de estos estimadores se mejoró aún más para todos
combinaciones de entradas densas y dispersas en conjuntos de datos float32 y float64, excepto
las combinaciones escaso-denso y denso-escaso para el euclidiano y el cuadrado
Métricas de distancia euclidiana.
Puede encontrar una lista detallada de los estimadores afectados en el
`registro de cambios <cambios_1_2>`.

